In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

np.random.seed(1)

%matplotlib inline
np.random.seed(1)

## Multivariate case

We are trying to minimise $\sum \xi_i^2$. This time $ f = Xw$ where $w$ is Dx1 and $X$ is NxD.

\begin{align}
\mathcal{L} & = \frac{1}{N} (y-Xw)^T(y-Xw) \\
\frac{\delta\mathcal{L}}{\delta w} & = -\frac{1}{N} 2\left(\frac{\delta f(X,w)}{\delta w}\right)^T(y-Xw) \\ 
& = -\frac{2}{N} \left(\frac{\delta f(X,w)}{\delta w}\right)^T\xi
\end{align}
where $\xi_i$ is the error term $y_i-f(X,w)$ and 
$$
\frac{\delta f(X,w)}{\delta w} = X
$$

Finally the weights can be updated as $w_{new} = w_{current} - \gamma \frac{\delta\mathcal{L}}{\delta w}$ where $\gamma$ is a learning rate between 0 and 1.

In [15]:
N = 1000
D = 5
X = 5*np.random.randn(N,D)
w_true = np.random.randn(D,1)
y = X.dot(w_true)
y_obs = y + np.random.randn(N,1)

In [16]:
model = LinearRegression(fit_intercept=False)
model.fit(X,y)
model.coef_.T

array([[ 0.06470447],
       [ 1.4400372 ],
       [-1.15309585],
       [ 0.63568265],
       [ 0.76527259]])

## Stochastic Gradient Descent

In [17]:
def dL_dw(X,e,w):
    return -2*e*X.T/len(X)

def u_choice(bet, un, sgC):
  u_norm = max(bet**2*un.T.dot(un) , sgC.T.dot(sgC))
  if u_norm ==  bet**2*un.T.dot(un) :
    return bet*un
  elif u_norm ==  abs(sgC.T.dot(sgC)) :
    return abs(sgC)
  else :
    return "Erreur in selecting u"

def loss_function(e):
    L = e.T.dot(e)/N
    return L

In [18]:

def adamax(X,y_obs,alpha=0.5, beta1 = 0.9, beta2=0.999,batch_size = 25, epoch = 10):
    #Initialisation des paramétres 
    N = len(X)
    D = len(X[0])
    # les conditions initiales
    w = np.random.randn(D,1)
    params = []
    loss = np.zeros((N,1))
    Loss = np.zeros((N,1))
    m = np.zeros((D,1))
    u = np.zeros((D,1))
    betai = 1
    for b in range(epoch):
        for i in range(batch_size):
            params.append(w)
            idx = np.random.choice(N,batch_size,replace=False)
            e = y_obs[idx] - X[idx].dot(w)
            #just for testing
            ##########################
            e_global = y_obs - X.dot(w)
            L = loss_function(e_global)
            ##########################
            sg = dL_dw(X[idx],e,w)
            m = beta1*m + (1-beta1)*sg
            u = u_choice(beta2,u,sg)
            betai = betai*beta1
            mHat = (m/(1-betai))
            w = w - alpha*mHat/u
            #calculer erreur (e) avec les nouvelles configs
            e = y_obs[idx] - X[idx].dot(w)
            loss[i] = e
            Loss[i] = L
    print("epoch",b, ": Loss = ", L, sep=" ")
        
    return params,loss,Loss

In [19]:
params, loss, Loss = adamax(X,y_obs)#Test avec X et y_obs

ValueError: operands could not be broadcast together with shapes (25,1) (5,25) 

In [20]:
plt.plot(loss)
plt.plot(Loss)

NameError: name 'loss' is not defined

In [7]:
print(params[-1])
Loss[-1]

[[0.299356]]


array([1.0273531])

In [8]:
# compare parameters side by side
np.hstack([params[-1],model.coef_.T])

array([[0.299356  , 0.23879586]])